In [5]:
import sys
!conda install --yes --prefix {sys.prefix} numpy

In [6]:
# open csv, save data into vectors
import csv
import numpy as np
with open('q2.csv', newline="") as csvfile:
    filereader = csv.reader(csvfile, delimiter=',')
    arrays = []
    for row in filereader:
        arrays.append(np.array(row))
    print(arrays)

[array(['age', 'course', 'likeStats', 'Y'], dtype='<U9'), array(['20', '1', '0', '0'], dtype='<U2'), array(['18', '1', '1', '0'], dtype='<U2'), array(['15', '0', '1', '1'], dtype='<U2'), array(['16', '0', '0', '1'], dtype='<U2'), array(['17', '1', '1', '0'], dtype='<U2'), array(['21', '1', '0', '0'], dtype='<U2'), array(['23', '1', '0', '1'], dtype='<U2'), array(['15', '1', '1', '0'], dtype='<U2'), array(['17', '0', '1', '1'], dtype='<U2'), array(['17', '1', '0', '0'], dtype='<U2')]


In [16]:
# (a) calculate the value of g(x) for each data point
theta = np.array([0.05,-3.0, 2.5])
theta = np.transpose(theta)
theta_naut = 0.3
print(theta)

gx = theta @ arrays[1][0:3] + theta_naut
print(gx)


[ 0.05 -3.    2.5 ]


UFuncTypeError: ufunc 'matmul' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')